_Import Necessary Libraries_ 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import json
import seaborn as sns
import os, time

import tensorflow as tf
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#########################
# DL Libraries
#########################
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomFlip, RandomZoom, Rescaling
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [ ]:
### set seed for producing same results
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

#### Load Training Data

In [ ]:
dir_path = '../input/cassava-leaf-disease-classification'
train_read = pd.read_csv(dir_path + "/train.csv", sep=',')
print ('dataframe shape: ', train_read.shape)
train_read.head(10)

#### Plot Class Distribution

In [ ]:
fig = plt.figure(figsize=(8, 6))

sns.set(font_scale=1.1)
label_count = sns.countplot(x='label', data=train_read, order = train_read['label'].value_counts().index)
label_count.set_xticklabels(label_count.get_xticklabels(), rotation=10)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Labels', fontsize=12)
plt.show()

Highly Class Imbalance Data : Class weight can be used later on. 
Sparse class or categroical class ? For simplicity I went along with sparse class, also going via categorical (one hot encoding the labels) path takes a lot of time (training the data).  

In [ ]:
with open(dir_path + '/label_num_to_disease_map.json') as f:
    labelnames = json.loads(f.read())
    labelnames = {int(k): v for k,v in labelnames.items()}

print(labelnames)
print(labelnames[4])

In [ ]:
### this was done to check sparse categorical cross entropy part 
### otherwise can be omitted 
train_read['label'] = train_read['label'].astype('string')
train_read.head(3)

#### Visualize Different Classes of Disease 

In [ ]:
train_im_path = dir_path + '/train_images/'


fig = plt.figure(figsize=(15, 10))
npics= 6

count = 1
image_list = train_read[train_read['label'] == str(list(labelnames.keys())[list(labelnames.values()).index('Healthy')])]['image_id'].sample(frac=1)[:npics].to_list()  
for i, img in enumerate(image_list):
    
    sample = os.path.join(train_im_path, img) 
    sample_img = Image.open(sample)   
    ax = fig.add_subplot(npics/2 , 3, count, xticks=[],yticks=[])   
    plt.imshow(sample_img)
    count +=1
fig.suptitle('All Healthy')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))
npics= 6

count = 1
image_list = train_read[train_read['label'] == str(list(labelnames.keys())[list(labelnames.values()).index('Cassava Bacterial Blight (CBB)')])]['image_id'].sample(frac=1)[:npics].to_list()  
for i, img in enumerate(image_list):
    
    sample = os.path.join(train_im_path, img) 
    sample_img = Image.open(sample)   
    ax = fig.add_subplot(npics/2 , 3, count, xticks=[],yticks=[])   
    plt.imshow(sample_img)
    count +=1
fig.suptitle('CBB')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))
npics= 6

count = 1
image_list = train_read[train_read['label'] == str(list(labelnames.keys())[list(labelnames.values()).index('Cassava Brown Streak Disease (CBSD)')])]['image_id'].sample(frac=1)[:npics].to_list()  
for i, img in enumerate(image_list):
    
    sample = os.path.join(train_im_path, img) 
    sample_img = Image.open(sample)   
    ax = fig.add_subplot(npics/2 , 3, count, xticks=[],yticks=[])   
    plt.imshow(sample_img)
    count +=1
fig.suptitle('CBSD')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))
npics= 6

count = 1
image_list = train_read[train_read['label'] == str(list(labelnames.keys())[list(labelnames.values()).index('Cassava Green Mottle (CGM)')])]['image_id'].sample(frac=1)[:npics].to_list()  
for i, img in enumerate(image_list):
    
    sample = os.path.join(train_im_path, img) 
    sample_img = Image.open(sample)   
    ax = fig.add_subplot(npics/2 , 3, count, xticks=[],yticks=[])   
    plt.imshow(sample_img)
    count +=1
fig.suptitle('CGM')
plt.tight_layout()
plt.show()

#### How to Find Any Differences 

I spent some time reading about these diseases, because to be frank just from the images it is almost impossible to distinguish between different classes. 
After reading about the diseases and common symptoms I got even more confused because few of them show similar symptoms.   

For CBSD, I found [this document](http://www.fao.org/3/CA2940EN/ca2940en.pdf) really helpful. 

For CGM, I foind [this webpage](https://apps.lucidcentral.org/ppp_v9/text/web_full/entities/cassava_green_mottle_068.htm) helpful. The point is to show the similar symptoms between different diseases. 

In [this document](https://assets.publishing.service.gov.uk/media/57a08d8140f0b649740018d4/R7563RootsEng.pdf) the authors discussed about how CMD varies from region to region. 

* Later on after classification, I reached around 86% accuracy on validation data and to me it is surprisingly good. Because at least to my untrained eyes, they all looked very similar. 

Also since most of the images are of different leaves, it is better to resize the data (maybe something like center crop ?), which will reduce the training time. 

In [ ]:
### check the image sizes if all are same or not 

im_name_lists = train_read['image_id'].tolist()
im_shape_x_lists = []
im_shape_y_lists = []
for i, img in enumerate(im_name_lists):
    sample = os.path.join(train_im_path, img) 
    sample_img = Image.open(sample)
    w, h = sample_img.size
#     im_shape_x_lists.append(sample_img.shape[0])
    im_shape_x_lists.append(w)
    im_shape_y_lists.append(h)
print ('check len: ', len(im_shape_x_lists), len(im_shape_y_lists))    

In [ ]:
fig = plt.figure(figsize=(6, 4))

fig.add_subplot(121)
plt.hist(im_shape_x_lists)
fig.add_subplot(122)
plt.hist(im_shape_y_lists)
plt.tight_layout()

print (set(im_shape_x_lists), set(im_shape_y_lists)) # 600 x 800 images 

These images are too big and it would exhaust the time and resource to process with original size. 
So we will resize the image depending on the pretrained model. 

I will use InceptionResNetV2 so we will resize the images to 299x299 (wxh)

In [ ]:
target_size = (300, 300)
input_shape = (300, 300, 3)
batch_size = 64

#### Create the Batch Generators  

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split=0.05)

train_generator = datagen.flow_from_dataframe(train_read,
                                              directory=train_im_path,
                                              x_col="image_id",
                                              y_col="label",
                                              target_size=target_size,
                                              batch_size=batch_size,
                                              class_mode="sparse",
                                              subset="training",)

val_generator = datagen.flow_from_dataframe(train_read,
                                            directory=train_im_path,
                                            x_col="image_id",
                                            y_col="label",
                                            target_size=target_size,
                                            batch_size=batch_size,
                                            class_mode="sparse", 
                                            subset="validation",)


In [ ]:
fig = plt.figure(figsize=(15, 10))
npics= 16
count = 1
for i in range(npics):
    x,y = val_generator.next()
    image = x[0].astype('uint8')
#     print (image.shape)
    label = y[0]  
    int_label = int(label)  
    ax = fig.add_subplot(npics/4 , 4, count, xticks=[],yticks=[])
    ax.set_title(labelnames[int_label], fontsize=10)  
    plt.imshow(image)
    count = count + 1  

plt.tight_layout()
plt.show()

### Build the Model Using Pre-Trained InceptionResNetV2

* We will also include augmentation as a model layer (Inspired from tf.data pipeline)
* Next target is to remove ImageDataGenerator completely.  

Added and tested Cosine Decay following [this notebook](https://www.kaggle.com/frlemarchand/efficientnet-aug-tf-keras-for-cassava-diseases) but the results are worse.  

In [ ]:
## This cell was used to compile the baseline model
### Cosine Decay was tested

class customCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    self.epoch = epoch + 1
    if self.epoch % 2 == 0:
      print (
          'epoch num {}, train loss: {}, validation loss: {}'.format(epoch, logs['loss'], logs['val_loss']))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1)


# epochs = 20
# decay_steps = int(round(len(train_read)/batch_size))*epochs

# cosine_decay = CosineDecay(initial_learning_rate=8e-4, decay_steps=decay_steps, alpha=0.3)


mcp_save = ModelCheckpoint(filepath="best_model_weights.h5",
                           save_best_only=True, save_weights_only=True, monitor='val_loss')




es = EarlyStopping(monitor="val_loss", patience=10,)


# targets are not one hot encoded but integers so we use sparse_categorical crossentropy
### later one targets were converted to one hot encoded labels 

# final_Efficient_model.compile(optimizer=adam, 
#                               loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False,
#                                                                            label_smoothing=0.001,), 
#                               metrics=['accuracy',])

In [ ]:
inception_resnet_v2 = InceptionResNetV2(
    include_top=False,
    weights="../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5",
    input_shape=input_shape,)

def build_model():
    inputs = Input(input_shape)
    
    x = preprocess_input(inputs)
    x = Rescaling(1./255)(x)
    
    ###### data augmentation layers
    x = RandomFlip()(x)
    x = RandomRotation(factor=0.3)(x)
    
    ###### InceptionResNetV2 + Some Top Layers
    x = BatchNormalization()(x)
    x = inception_resnet_v2(x)

    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (1, 1), activation=LeakyReLU())(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dropout(0.75)(x)

    x = Dense(256, activation=LeakyReLU())(x)
    x = Dropout(0.80)(x)
    x = BatchNormalization()(x)
    
    outputs = Dense(5, activation="softmax")(x)
    
    model = Model(inputs, outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss="sparse_categorical_crossentropy", 
                  metrics=["accuracy"])
    
    return model


In [ ]:
model = build_model()
model.summary()

##### Add Class Weight 


Treatment for Unbalanced Data 

* Try Multi-class Focal Loss later 

**Class_Weight** worsens the performace, so omitted from newer versions. 

In [ ]:
labels_int = pd.to_numeric(train_read['label'], errors='coerce')
print (type(labels_int))
labels_int_arr = labels_int.to_numpy()
# np.unique(labels_int_arr)
# labels_int_arr.shape

from sklearn.utils import class_weight
mod_class_weights = class_weight.compute_class_weight('balanced', 
                                                      np.unique(labels_int_arr), labels_int_arr)

print (mod_class_weights)
print (dict(enumerate(mod_class_weights)))

In [ ]:
start_time = time.time()
history = model.fit(train_generator, 
                    validation_data=val_generator, 
                    epochs=100, 
                    callbacks=[mcp_save, es, reduce_lr])

end_time = time.time()

In [ ]:
print ('total time taken: in Minutes', (end_time-start_time)/60.)

#### Training and Validation Curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig = plt.figure(figsize=(15, 5))
fig.add_subplot(121)

plt.plot(epochs, acc, linestyle='--', label = "Training acc")
plt.plot(epochs, val_acc, linestyle='-.', label = "Validation acc")
plt.title("Training and validation acc")
plt.legend()

fig.add_subplot(122)
plt.plot(epochs, loss, linestyle='--', label = "Training loss", alpha=0.8)
plt.plot(epochs, val_loss, linestyle='-.', label = "Validation loss", alpha=0.6)
plt.title("Training and validation loss")
plt.legend()

plt.show()


In [ ]:
model.evaluate(val_generator)

In [ ]:
### check if the model is still fine after loading the trained weights
model.load_weights("best_model_weights.h5")
model.evaluate(val_generator)

#### Preparing for Submission

In [ ]:
submission_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
submission_df.head()


In [ ]:
preds = []
# preds_no_argmax = []


test_images = os.listdir('/kaggle/input/cassava-leaf-disease-classification/test_images/')
preds = []

for i in test_images:
    image = Image.open(f'/kaggle/input/cassava-leaf-disease-classification/test_images/{i}')
    image = image.resize(target_size)
    image = np.expand_dims(image, axis=0)
    preds.append(np.argmax(model.predict(image)))

In [ ]:
df_sub = pd.DataFrame({'image_id': test_images, 'label': preds})
df_sub.head()

In [ ]:
df_sub.to_csv("submission.csv", index=None)

#### Plot the Confusion Matrix (Validation Data)

In [ ]:
class_types = list(labelnames.values())
print (class_types)

In [ ]:
### try to plot the confusion matrix

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns



def conf_matrix(test_lab, predictions): 
    ''' Plots conf. matrix and classification report '''
    cm=confusion_matrix(test_lab, np.argmax(np.round(predictions), axis=1))
    print("Classification Report:\n")
    cr=classification_report(test_lab,
                                np.argmax(np.round(predictions), axis=1), 
                                target_names=[class_types[i] for i in range(len(class_types))])
    print(cr)
    plt.figure(figsize=(8,8))
    sns_hmp = sns.heatmap(cm, annot=True, xticklabels = [class_types[i] for i in range(len(class_types))], 
                yticklabels = [class_types[i] for i in range(len(class_types))], fmt="d")
    fig = sns_hmp.get_figure()

In [ ]:
npics = 600 # try 500 examples 
valid_preds = []
all_valid_ims = []
all_valid_labels = []
for _ in range(npics):
    x,y = val_generator.next()
    image = x[0].astype('uint8')
    label = y[0]
    all_valid_labels.append(label)
    image = np.expand_dims(image, axis = 0)
    all_valid_ims.append(image)


In [ ]:
all_valid_ims_arr = np.array(all_valid_ims)
all_valid_labels_arr = np.array(all_valid_labels)


all_valid_ims_arr = np.reshape(all_valid_ims_arr, (600, 300, 300, 3))

print ('check shapes now: ', all_valid_ims_arr.shape, all_valid_labels_arr.shape)


In [ ]:
pred_class_InceptResV2 = model.predict(all_valid_ims_arr)
print ('check shape of preds: ', pred_class_InceptResV2.shape)

In [ ]:
conf_matrix(np.int32(all_valid_labels_arr), pred_class_InceptResV2)